In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype=np.float32)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
array([[[-0.4278658, -0.4605183]]], dtype=float32)


### Unfolding to n sequences
sequence_length: 한 번에 Cell을 몇개나 만들것인지

```
# hidden_size: 2, sequence_length: 5
shape=(1,5,2): [[[x,x], [x,x], [x,x], [x,x], [x,x]]]
```

In [4]:
with tf.variable_scope('two_sequences') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    # Sequence: 5
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.7602298 , -0.26838937],
        [-0.11971088, -0.3753212 ],
        [ 0.53579473, -0.08472025],
        [-0.09884749, -0.24635318],
        [-0.28428522, -0.334213  ]]], dtype=float32)


### Batching input
한번에 네트워크에 넘겨주는 데이터의 수를 하나씩 주는것 보다 여러개 주는게 좋음.
1000개의 데이터를 batch_size=10 으로 넘겨준다고 하면, 총 10개씩 배치로 그룹을 이루어서 들어가게 되고, 총 100개의 step을 통해 1epoch을 도는 것임.

```
# hidden_size: 2
# sequence_length: 5
# batch_size: 3
shape=(3,5,2): [[[x, x], [x,x], [x,x], [x,x], [x,x]],
                [[x, x], [x,x], [x,x], [x,x], [x,x]],
                [[x, x], [x,x], [x,x], [x,x], [x,x]]]
```

In [5]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h,e,l,l,o],
                      [e,o,l,l,o],
                      [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
array([[[ 0.10256949, -0.04174667],
        [ 0.03061712, -0.13762395],
        [-0.03607963, -0.19260055],
        [-0.08959933, -0.22006084],
        [ 0.00539434, -0.26254502]],

       [[-0.06502195, -0.09742635],
        [ 0.02122071, -0.17587155],
        [-0.03115345, -0.20066279],
        [-0.08621308, -0.22673753],
        [ 0.0065

In [7]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h,e,l,l,o],
                      [e,o,l,l,l],
                      [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
array([[[-0.13821399, -0.10808922],
        [ 0.00656023, -0.18700112],
        [ 0.1618441 , -0.06883518],
        [ 0.26723063,  0.03702846],
        [ 0.14864516,  0.03364095]],

       [[ 0.08764657, -0.03722832],
        [-0.02305177, -0.01738983],
        [ 0.13327485,  0.05913957],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.1481339 ,  0.0782133 ],
        [ 0.25236756,  0.14670895],
        [ 0.20608513,  0.11380293],
        [ 0.2141692 ,  

In [8]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h,e,l,l,o],
                      [e,o,l,l,l],
                      [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state=initial_state, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.11347992, -0.12118694],
        [ 0.07579508, -0.01807293],
        [-0.0420468 , -0.03566688],
        [-0.11394374, -0.04091628],
        [-0.12990601, -0.1025131 ]],

       [[ 0.02512887,  0.07370635],
        [-0.02262686, -0.01548756],
        [-0.09903497, -0.02604243],
        [-0.14769112, -0.02885182],
        [-0.17687917, -0.0273458 ]],

       [[-0.08606549, -0.01538836],
        [-0.13936135, -0.02119916],
        [-0.06149898,  0.06963572],
        [-0.00673014,  0.13002405],
        [-0.08281367,  0.09392559]]], dtype=float32)
